# 탐색적 데이터 분석 1

* ai허브(aihub.or.kr)의 한국어 방언 발화 데이터셋을 살펴보고 전처리하는 과정
* 1차 시도

In [1]:
import pandas as pd
import numpy as np
import pathlib
import random
import re

RANDOM_SEED = 19439
random.seed(RANDOM_SEED)

In [3]:
df = pd.read_csv("datas/output/jeonla_dialect_data.csv", index_col=0)
df.head()

,방언,표준어,출처 파일
0,나는 다이어트에 대해서 말하고 싶어.,나는 다이어트에 대해서 말하고 싶어.,DJDD20000001.json
1,하,하,DJDD20000001.json
2,우리 집은 내가 봤을 때 살찔 수밖에 없는 구조를 가지고 있어.,우리 집은 내가 봤을 때 살찔 수밖에 없는 구조를 가지고 있어.,DJDD20000001.json
3,다이어트를 해야 되는 사람은 애초에 야식을 많이 먹거나 아 어제도 야식을 먹었거든?,다이어트를 해야 되는 사람은 애초에 야식을 많이 먹거나 아 어제도 야식을 먹었거든?,DJDD20000001.json
4,{laughing},{laughing},DJDD20000001.json


In [18]:
len(df)

1992101

총 199만 2101개의 발화 데이터 쌍이 존재한다.

하지만 이 데이터에는 결함이 많음을 직접 파일을 열어 확인했다.

대표적으로 (())과 같이 텍스트 상에서는 의미가 없는 괄호가 들어간 데이터와 {laughing}으로 표현된 웃는 소리, (방언)/(표준어)로 같이 적은 문장이 있었다.

가장 큰 문제는 방언임에도 불구하고 이를 표준어에 그대로 적어놓은 경우이다. 이 경우는 프로그램으로 제거하기가 어렵다.

이를 정규식을 이용해 걸러낼 계획이다.

## (()) 문자열 제거

In [19]:
df[df["방언"].str.contains("\(\(\)\)")|df["표준어"].str.contains("\(\(\)\)")]

,방언,표준어,출처 파일


표준어나 방언에 (())가 들어간 셋이 총 39016개 존재했다.

이들에 대해서는 간단히 (())를 제거하는 조치를 취하기로 하였다.

In [20]:
df["방언"] = df["방언"].str.replace(r"\(\(\)\)", '', regex=True)
df["표준어"] = df["표준어"].str.replace(r"\(\(\)\)", '', regex=True)

df[df["방언"].str.contains("\(\(\)\)")|df["표준어"].str.contains("\(\(\)\)")]

,방언,표준어,출처 파일


의도대로 (())가 포함된 데이터가 사라졌음을 확인할 수 있다.

## 괄호 안에 문자열이 들어간 경우 처리

### ((문자열)) 처리

In [21]:
df[df["방언"].str.contains("\(.*\)")]

,방언,표준어,출처 파일
152,그냥 적게 먹고 많이 ((xx하는)) 거고 그냥 그렇게 하면은 되는데 진짜 그런 광...,그냥 적게 먹고 많이 ((xx하는)) 거고 그냥 그렇게 하면은 되는데 진짜 그런 광...,DJDD20000001.json
176,약간 나는 운동(하믄서)/(하면서) 스트레스 풀라는 것도 있긴 한데,약간 나는 운동(하믄서)/(하면서) 스트레스 풀라는 것도 있긴 한데,DJDD20000001.json
242,막 포만감도 막 엄청 많지 않으니까 그래서 ((xx xxx도)) 나름대로 높아지고,막 포만감도 막 엄청 많지 않으니까 그래서 ((xx xxx도)) 나름대로 높아지고,DJDD20000001.json
281,약간 이 생각도 많이 ((들어)),약간 이 생각도 많이 ((들어)),DJDD20000001.json
330,근데 확실히 다이어트 평생 ((해야xx)) 느낀 게 우리 엄마도 살이 찌더라고 {l...,근데 확실히 다이어트 평생 ((해야xx)) 느낀 게 우리 엄마도 살이 찌더라고 {l...,DJDD20000001.json
...,...,...,...
1991447,자제(할려고)/(하려고) 쫌 그렇게 신경을 쓰기는 해.,자제(할려고)/(하려고) 조금 그렇게 신경을 쓰기는 해.,DJSX20004851.json
1991464,((무슨))의미가 있겠느냐 술이야 뭐 쫌 쭐이면서 점점,((무슨))의미가 있겠느냐 술이야 뭐 조금 쭐이면서 점점,DJSX20004851.json
1991488,요즘에 다 시국이 시국인지라(잉)/(응) 다,요즘에 다 시국이 시국인지라(잉)/(응) 다,DJSX20004851.json
1991508,혼자서 할 수 있는 ((운동)) 하고 집에 뭐~,혼자서 할 수 있는 ((운동)) 하고 집에 뭐~,DJSX20004851.json


눈으로 확인할 수 있는 경우가 두 가지 있다. (()) 안에 글자가 들어간 경우와 (방언)/(표준어) 쌍이 들어있는 경우이다.

(()) 안에 글자가 들은 경우는 그 안의 글자를 빼내고, (방언)/(표준어) 쌍의 경우는 각 열에 맞추어 적절한 단어를 선택하여 대입할 계획이다.

다만 두 경우를 합쳐 만 7천 개 정도이니 전체 중 비율은 0.9%에 불과하다. 이는 다행스러운 일이다.

In [22]:
df[df["방언"].str.contains("\(\(.*\)\)")]

,방언,표준어,출처 파일
152,그냥 적게 먹고 많이 ((xx하는)) 거고 그냥 그렇게 하면은 되는데 진짜 그런 광...,그냥 적게 먹고 많이 ((xx하는)) 거고 그냥 그렇게 하면은 되는데 진짜 그런 광...,DJDD20000001.json
242,막 포만감도 막 엄청 많지 않으니까 그래서 ((xx xxx도)) 나름대로 높아지고,막 포만감도 막 엄청 많지 않으니까 그래서 ((xx xxx도)) 나름대로 높아지고,DJDD20000001.json
281,약간 이 생각도 많이 ((들어)),약간 이 생각도 많이 ((들어)),DJDD20000001.json
330,근데 확실히 다이어트 평생 ((해야xx)) 느낀 게 우리 엄마도 살이 찌더라고 {l...,근데 확실히 다이어트 평생 ((해야xx)) 느낀 게 우리 엄마도 살이 찌더라고 {l...,DJDD20000001.json
342,나는 다이어트는 진짜 ((그래도)) 진짜 아니면 평생 그냥 좀 뚱뚱하고 통통하게 살던가,나는 다이어트는 진짜 ((그래도)) 진짜 아니면 평생 그냥 좀 뚱뚱하고 통통하게 살던가,DJDD20000001.json
...,...,...,...
1990837,어~ 거의 시내 쪽에서 뭐~~ 콜로세움 뭐~ ((그케이케)) 뭐~ 거기 시내,어~ 거의 시내 쪽에서 뭐~~ 콜로세움 뭐~ ((그케이케)) 뭐~ 거기 시내,DJSX20004850.json
1991374,그게 의지 ((그래)) 의지일 수도 있는데,그게 의지 ((그래)) 의지일 수도 있는데,DJSX20004851.json
1991423,자기 입장에서도 난감하고 물론 ((암도다)) 몇 달 후에,자기 입장에서도 난감하고 물론 ((암도다)) 몇 달 후에,DJSX20004851.json
1991464,((무슨))의미가 있겠느냐 술이야 뭐 쫌 쭐이면서 점점,((무슨))의미가 있겠느냐 술이야 뭐 조금 쭐이면서 점점,DJSX20004851.json


In [27]:
df["방언"].str.extract("\(\((.*)\)\)", expand=False).dropna()

152           xx하는
242        xx xxx도
281             들어
330           해야xx
342            그래도
            ...   
1990837       그케이케
1991374         그래
1991423        암도다
1991464         무슨
1991508         운동
Name: 방언, Length: 12452, dtype: object

In [35]:
phrase_words = df["방언"].str.extract(r"\(\((.*)\)\)", expand=False).dropna()
df.loc[phrase_words.index, "방언"]

152        그냥 적게 먹고 많이 ((xx하는)) 거고 그냥 그렇게 하면은 되는데 진짜 그런 광...
242            막 포만감도 막 엄청 많지 않으니까 그래서 ((xx xxx도)) 나름대로 높아지고
281                                       약간 이 생각도 많이 ((들어))
330        근데 확실히 다이어트 평생 ((해야xx)) 느낀 게 우리 엄마도 살이 찌더라고 {l...
342         나는 다이어트는 진짜 ((그래도)) 진짜 아니면 평생 그냥 좀 뚱뚱하고 통통하게 살던가
                                 ...                        
1990837           어~ 거의 시내 쪽에서 뭐~~ 콜로세움 뭐~ ((그케이케)) 뭐~ 거기 시내
1991374                              그게 의지 ((그래)) 의지일 수도 있는데
1991423                      자기 입장에서도 난감하고 물론 ((암도다)) 몇 달 후에
1991464                       ((무슨))의미가 있겠느냐 술이야 뭐 쫌 쭐이면서 점점
1991508                           혼자서 할 수 있는 ((운동)) 하고 집에 뭐~
Name: 방언, Length: 12452, dtype: object

In [36]:
phrase_words = df["방언"].str.extract(r"\(\((.*)\)\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "방언"]
    sentence = re.sub(r"\(\(.*\)\)", phrase_words[i], sentence)
    df.loc[i, "방언"] = sentence

df.loc[phrase_words.index, "방언"]

152        그냥 적게 먹고 많이 xx하는 거고 그냥 그렇게 하면은 되는데 진짜 그런 광고 같은...
242                막 포만감도 막 엄청 많지 않으니까 그래서 xx xxx도 나름대로 높아지고
281                                           약간 이 생각도 많이 들어
330        근데 확실히 다이어트 평생 해야xx 느낀 게 우리 엄마도 살이 찌더라고 {laugh...
342             나는 다이어트는 진짜 그래도 진짜 아니면 평생 그냥 좀 뚱뚱하고 통통하게 살던가
                                 ...                        
1990837               어~ 거의 시내 쪽에서 뭐~~ 콜로세움 뭐~ 그케이케 뭐~ 거기 시내
1991374                                  그게 의지 그래 의지일 수도 있는데
1991423                          자기 입장에서도 난감하고 물론 암도다 몇 달 후에
1991464                           무슨의미가 있겠느냐 술이야 뭐 쫌 쭐이면서 점점
1991508                               혼자서 할 수 있는 운동 하고 집에 뭐~
Name: 방언, Length: 12452, dtype: object

In [37]:
# 표준어 column에 대해서도 같은 과정을 반복한다.
phrase_words = df["표준어"].str.extract(r"\(\((.*)\)\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "표준어"]
    sentence = re.sub(r"\(\(.*\)\)", phrase_words[i], sentence)
    df.loc[i, "표준어"] = sentence

df.loc[phrase_words.index, "표준어"]

152        그냥 적게 먹고 많이 xx하는 거고 그냥 그렇게 하면은 되는데 진짜 그런 광고 같은...
242                막 포만감도 막 엄청 많지 않으니까 그래서 xx xxx도 나름대로 높아지고
281                                           약간 이 생각도 많이 들어
330        근데 확실히 다이어트 평생 해야xx 느낀 게 우리 엄마도 살이 찌더라고 {laugh...
342             나는 다이어트는 진짜 그래도 진짜 아니면 평생 그냥 좀 뚱뚱하고 통통하게 살던가
                                 ...                        
1990837               어~ 거의 시내 쪽에서 뭐~~ 콜로세움 뭐~ 그케이케 뭐~ 거기 시내
1991374                                  그게 의지 그래 의지일 수도 있는데
1991423                          자기 입장에서도 난감하고 물론 암도다 몇 달 후에
1991464                          무슨의미가 있겠느냐 술이야 뭐 조금 쭐이면서 점점
1991508                               혼자서 할 수 있는 운동 하고 집에 뭐~
Name: 표준어, Length: 12448, dtype: object

### (방언)/(표준어) 쌍 처리

In [38]:
df[df["방언"].str.contains("\(.*\)\/\(.*\)")]

,방언,표준어,출처 파일
176,약간 나는 운동(하믄서)/(하면서) 스트레스 풀라는 것도 있긴 한데,약간 나는 운동(하믄서)/(하면서) 스트레스 풀라는 것도 있긴 한데,DJDD20000001.json
8812,근데 우리 아빠는 오십 살(이냐)/(이잖아) 그니까 그래서 엄마 더 그런 거 있어.,근데 우리 아빠는 오십 살(이냐)/(이잖아) 그러니까 그래서 엄마 더 그런 거 있어.,DJDD20000019.json
8841,그리고 엄마도 그 할머니 병원에 이틀 있었다고 서로 스트레스 받아(갖고)/(가지고),그리고 엄마도 그 할머니 병원에 이틀 있었다고 서로 스트레스 받아(갖고)/(가지고),DJDD20000019.json
9048,엄마가 딱 싫어하는 스타일(이여)/(이야),엄마가 딱 싫어하는 스타일(이여)/(이야),DJDD20000019.json
12284,이십일 편인데 이틀 몰아(가꼬)/(가지고) 걍 미친듯이 봤단 말야 하아 흡입력 개...,이십일 편인데 이틀 몰아(가꼬)/(가지고) 그냥 미친듯이 봤단 말야 하아 흡입력 개...,DJDD20000028.json
...,...,...,...
1991378,멋있어 보이고 그래서 나도 담배 피워(볼려고)/(보려고) 시도를 해봤는데,멋있어 보이고 그래서 나도 담배 피워(볼려고)/(보려고) 시도를 해봤는데,DJSX20004851.json
1991418,이렇게 또 뇌 이런(잉)/(응) 심근 경색,이렇게 또 뇌 이런(잉)/(응) 심근 경색,DJSX20004851.json
1991447,자제(할려고)/(하려고) 쫌 그렇게 신경을 쓰기는 해.,자제(할려고)/(하려고) 조금 그렇게 신경을 쓰기는 해.,DJSX20004851.json
1991488,요즘에 다 시국이 시국인지라(잉)/(응) 다,요즘에 다 시국이 시국인지라(잉)/(응) 다,DJSX20004851.json


In [41]:
phrase_words = df["방언"].str.extract(r"\((.*)\)\/\(.*\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "방언"]
    sentence = re.sub(r"\(.*\)\/\(.*\)", phrase_words[i], sentence)
    df.loc[i, "방언"] = sentence

df.loc[phrase_words.index, "방언"]

176                      약간 나는 운동하믄서 스트레스 풀라는 것도 있긴 한데
8812            근데 우리 아빠는 오십 살이냐 그니까 그래서 엄마 더 그런 거 있어.
8841            그리고 엄마도 그 할머니 병원에 이틀 있었다고 서로 스트레스 받아갖고
9048                                  엄마가 딱 싫어하는 스타일이여
12284      이십일 편인데 이틀 몰아가꼬 걍  미친듯이 봤단 말야 하아 흡입력 개 쩔어 걍
                              ...                     
1991378               멋있어 보이고 그래서 나도 담배 피워볼려고 시도를 해봤는데
1991418                             이렇게 또 뇌 이런잉 심근 경색 
1991447                         자제할려고 쫌 그렇게 신경을 쓰기는 해.
1991488                            요즘에 다 시국이 시국인지라잉 다 
1991765                     투자자금내에 -맻개- 맻개만 딱 이렇게 우량주를
Name: 방언, Length: 4457, dtype: object

In [42]:
phrase_words = df["표준어"].str.extract(r"\(.*\)\/\((.*)\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "표준어"]
    sentence = re.sub(r"\(.*\)\/\(.*\)", phrase_words[i], sentence)
    df.loc[i, "표준어"] = sentence

df.loc[phrase_words.index, "표준어"]

176                        약간 나는 운동하면서 스트레스 풀라는 것도 있긴 한데
8812            근데 우리 아빠는 오십 살이잖아 그러니까 그래서 엄마 더 그런 거 있어.
8841             그리고 엄마도 그 할머니 병원에 이틀 있었다고 서로 스트레스 받아가지고
9048                                    엄마가 딱 싫어하는 스타일이야
12284      이십일 편인데 이틀 몰아가지고 그냥 미친듯이 봤단 말야 하아 흡입력 개 쩔어 그냥
                               ...                      
1991378                 멋있어 보이고 그래서 나도 담배 피워보려고 시도를 해봤는데
1991418                               이렇게 또 뇌 이런응 심근 경색 
1991447                          자제하려고 조금 그렇게 신경을 쓰기는 해.
1991488                              요즘에 다 시국이 시국인지라응 다 
1991765                       투자자금내에 -몇개- 몇개만 딱 이렇게 우량주를
Name: 표준어, Length: 4457, dtype: object

## {laughing} 태그 제거

In [44]:
laugh_index = df["방언"].str.contains("\{laughing\}")
df[laugh_index]

,방언,표준어,출처 파일
4,{laughing},{laughing},DJDD20000001.json
27,약간 거울 봤을 때 약간 내가 괜찮아 보여 {laughing},약간 거울 봤을 때 약간 내가 괜찮아 보여 {laughing},DJDD20000001.json
70,그러는 게 제일 쉬우니까 {laughing},그러는 게 제일 쉬우니까 {laughing},DJDD20000001.json
71,그게 제일 만만하잖아. {laughing},그게 제일 만만하잖아. {laughing},DJDD20000001.json
77,그럼 거울 보면서 만족감을 느끼는 거지.{laughing},그럼 거울 보면서 만족감을 느끼는 거지.{laughing},DJDD20000001.json
...,...,...,...
1991939,내 김치 맵다고 아예 맵지 않은 김치를 사서 먹거든 너무 너무 편하고 좋더라 {la...,내 김치 맵다고 아예 맵지 않은 김치를 사서 먹거든 너무 너무 편하고 좋더라 {la...,DJSX20004853.json
1991946,미쳤다고 그러잖아 그걸 {laughing} 안 맛있어요.,미쳤다고 그러잖아 그걸 {laughing} 안 맛있어요.,DJSX20004853.json
1992042,어떻게 이렇게 같이 살아 주는지 나도 참 대단하다 {laughing},어떻게 이렇게 같이 살아 주는지 나도 참 대단하다 {laughing},DJSX20004853.json
1992072,구제 라나 뭐라나 만원 주고 샀다는데 내가 어이가 없어 한참 웃었다 야 {laugh...,구제 라나 뭐라나 만원 주고 샀다는데 내가 어이가 없어 한참 웃었다 야 {laugh...,DJSX20004853.json


In [50]:
df["방언"] = df["방언"].str.replace(r"\{laughing\}", "", regex=True)
df[laugh_index]

,방언,표준어,출처 파일
4,,{laughing},DJDD20000001.json
27,약간 거울 봤을 때 약간 내가 괜찮아 보여,약간 거울 봤을 때 약간 내가 괜찮아 보여 {laughing},DJDD20000001.json
70,그러는 게 제일 쉬우니까,그러는 게 제일 쉬우니까 {laughing},DJDD20000001.json
71,그게 제일 만만하잖아.,그게 제일 만만하잖아. {laughing},DJDD20000001.json
77,그럼 거울 보면서 만족감을 느끼는 거지.,그럼 거울 보면서 만족감을 느끼는 거지.{laughing},DJDD20000001.json
...,...,...,...
1991939,내 김치 맵다고 아예 맵지 않은 김치를 사서 먹거든 너무 너무 편하고 좋더라,내 김치 맵다고 아예 맵지 않은 김치를 사서 먹거든 너무 너무 편하고 좋더라 {la...,DJSX20004853.json
1991946,미쳤다고 그러잖아 그걸 안 맛있어요.,미쳤다고 그러잖아 그걸 {laughing} 안 맛있어요.,DJSX20004853.json
1992042,어떻게 이렇게 같이 살아 주는지 나도 참 대단하다,어떻게 이렇게 같이 살아 주는지 나도 참 대단하다 {laughing},DJSX20004853.json
1992072,구제 라나 뭐라나 만원 주고 샀다는데 내가 어이가 없어 한참 웃었다 야,구제 라나 뭐라나 만원 주고 샀다는데 내가 어이가 없어 한참 웃었다 야 {laugh...,DJSX20004853.json


In [51]:
df["표준어"] = df["표준어"].str.replace(r"\{laughing\}", "", regex=True)
df[laugh_index]

,방언,표준어,출처 파일
4,,,DJDD20000001.json
27,약간 거울 봤을 때 약간 내가 괜찮아 보여,약간 거울 봤을 때 약간 내가 괜찮아 보여,DJDD20000001.json
70,그러는 게 제일 쉬우니까,그러는 게 제일 쉬우니까,DJDD20000001.json
71,그게 제일 만만하잖아.,그게 제일 만만하잖아.,DJDD20000001.json
77,그럼 거울 보면서 만족감을 느끼는 거지.,그럼 거울 보면서 만족감을 느끼는 거지.,DJDD20000001.json
...,...,...,...
1991939,내 김치 맵다고 아예 맵지 않은 김치를 사서 먹거든 너무 너무 편하고 좋더라,내 김치 맵다고 아예 맵지 않은 김치를 사서 먹거든 너무 너무 편하고 좋더라,DJSX20004853.json
1991946,미쳤다고 그러잖아 그걸 안 맛있어요.,미쳤다고 그러잖아 그걸 안 맛있어요.,DJSX20004853.json
1992042,어떻게 이렇게 같이 살아 주는지 나도 참 대단하다,어떻게 이렇게 같이 살아 주는지 나도 참 대단하다,DJSX20004853.json
1992072,구제 라나 뭐라나 만원 주고 샀다는데 내가 어이가 없어 한참 웃었다 야,구제 라나 뭐라나 만원 주고 샀다는데 내가 어이가 없어 한참 웃었다 야,DJSX20004853.json


## 빈 문자열 제거

In [58]:
is_empty_dialect = np.where(df["방언"] == '', True, False).tolist()
is_empty_standard = np.where(df["표준어"] == '', True, False).tolist()
is_empty = [dia or sta for dia, sta in zip(is_empty_dialect, is_empty_standard)]
df.loc[is_empty]

,방언,표준어,출처 파일
4,,,DJDD20000001.json
89,,,DJDD20000001.json
1272,,,DJDD20000004.json
1531,,,DJDD20000004.json
3431,,,DJDD20000008.json
...,...,...,...
1985289,,,DJSX20004826.json
1986525,,,DJSX20004837.json
1989326,,,DJSX20004845.json
1989397,,,DJSX20004845.json


총 3453개의 데이터가 빈 문자열을 가진다. 

이들을 제거할 필요가 있다.

In [61]:
empty_index = df.loc[is_empty].index
df = df.drop(empty_index)

1차적인 전처리가 끝았으니 우선 데이터를 출력한다.

In [62]:
df.to_csv("datas/output/jeonla_dialect_data_processed_1.csv")

## 표준어에 방언 텍스트가 그대로 들어간 경우

In [2]:
df = pd.read_csv("datas/output/jeonla_dialect_data_processed_1.csv", index_col=0)
df.loc[438064]

방언       아직까지는 큰 이상은 없는 거 같애 긍까 너는 지금 인제 막 그런 증상들이 나온다니까
표준어      아직까지는 큰 이상은 없는 거 같애 긍까 너는 지금 인제 막 그런 증상들이 나온다니까
출처 파일                                  DJDD20001725.json
Name: 438064, dtype: object

위의 경우 '긍까'라는 방언이 표준어에 그대로 들어가 있는 것을 확인할 수 있다.

이는 원본 파일에서도 나타나는 문제로, 데이터 자체의 문제이다. 이를 처리하고는 싶으나 이러한 오류는 사람이 직접 확인해야만 한다.

따라서 오류를 직접 확인해서 수정하는 작업을 하는 것은 모델 구축 기간이 짧아 불가능하다.

## -(문자열)- 처리

데이터가 발화 데이터이다 보니 화자가 말을 더듬거나 고쳐 말할 때가 있다. 이때 고쳐 말한 부분을 -(문자열)- 방식으로 표현하였다.

이런 문자열은 완전히 삭제해 버리는 게 적절하다.

In [4]:
df.loc[df["방언"].str.contains(r"\-.*\-")]

,방언,표준어,출처 파일
115,근데 어쩔 수 없이 해야지 약간 이런 생각 때문에 했는데 나는 솔직히 피티가 더 -...,근데 어쩔 수 없이 해야지 약간 이런 생각 때문에 했는데 나는 솔직히 피티가 더 -...,DJDD20000001.json
209,아 -그런- 진짜로 그런 경우도 많이 봤어.,아 -그런- 진짜로 그런 경우도 많이 봤어.,DJDD20000001.json
387,-그- 그 근육 늘리기가 진짜 힘들더라고,-그- 그 근육 늘리기가 진짜 힘들더라고,DJDD20000001.json
455,맞어 맞어 나도 -사- 사실 막 연애를 보면서 진짜 완전 불타오르는 사랑 그런 거 ...,맞어 맞어 나도 -사- 사실 막 연애를 보면서 진짜 완전 불타오르는 사랑 그런 거 ...,DJDD20000002.json
458,아 연애 저렇게 연애하면 진짜 -죽어나- 다 죽어나가겠구나 약간 이 생각 했어.,아 연애 저렇게 연애하면 진짜 -죽어나- 다 죽어나가겠구나 약간 이 생각 했어.,DJDD20000002.json
...,...,...,...
1991880,본인이 끓였대 그래서 뭐 -미- 미역국에다가 북어를 넣어서,본인이 끓였대 그래서 뭐 -미- 미역국에다가 북어를 넣어서,DJSX20004853.json
1991889,예전에 -안- 안 좋았던 기억에 좀 많이 있어 가지고,예전에 -안- 안 좋았던 기억에 좀 많이 있어 가지고,DJSX20004853.json
1991956,점심때도 너무 늦게 좀 일하다 보니까 너무 -늦-늦게 나와 가지고,점심때도 너무 늦게 좀 일하다 보니까 너무 -늦-늦게 나와 가지고,DJSX20004853.json
1991960,그렇게 기다리고 -있었- 있었던 거 아니야?,그렇게 기다리고 -있었- 있었던 거 아니야?,DJSX20004853.json


In [5]:
stutter_sentence = df["방언"].str.contains(r"\-.*\-")
df["방언"] = df["방언"].str.replace(r"\-.*\-", "", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\-.*\-", "", regex=True)

df.loc[stutter_sentence]

C:\Users\wnsgh\AppData\Local\Temp\ipykernel_14452\656797981.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["방언"] = df["방언"].str.replace(r"\-.*\-", "")
C:\Users\wnsgh\AppData\Local\Temp\ipykernel_14452\656797981.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["표준어"] = df["표준어"].str.replace(r"\-.*\-", "")


,방언,표준어,출처 파일
115,근데 어쩔 수 없이 해야지 약간 이런 생각 때문에 했는데 나는 솔직히 피티가 더 ...,근데 어쩔 수 없이 해야지 약간 이런 생각 때문에 했는데 나는 솔직히 피티가 더 ...,DJDD20000001.json
209,아 진짜로 그런 경우도 많이 봤어.,아 진짜로 그런 경우도 많이 봤어.,DJDD20000001.json
387,그 근육 늘리기가 진짜 힘들더라고,그 근육 늘리기가 진짜 힘들더라고,DJDD20000001.json
455,맞어 맞어 나도 사실 막 연애를 보면서 진짜 완전 불타오르는 사랑 그런 거 해가지고,맞어 맞어 나도 사실 막 연애를 보면서 진짜 완전 불타오르는 사랑 그런 거 해가지고,DJDD20000002.json
458,아 연애 저렇게 연애하면 진짜 다 죽어나가겠구나 약간 이 생각 했어.,아 연애 저렇게 연애하면 진짜 다 죽어나가겠구나 약간 이 생각 했어.,DJDD20000002.json
...,...,...,...
1991880,본인이 끓였대 그래서 뭐 미역국에다가 북어를 넣어서,본인이 끓였대 그래서 뭐 미역국에다가 북어를 넣어서,DJSX20004853.json
1991889,예전에 안 좋았던 기억에 좀 많이 있어 가지고,예전에 안 좋았던 기억에 좀 많이 있어 가지고,DJSX20004853.json
1991956,점심때도 너무 늦게 좀 일하다 보니까 너무 늦게 나와 가지고,점심때도 너무 늦게 좀 일하다 보니까 너무 늦게 나와 가지고,DJSX20004853.json
1991960,그렇게 기다리고 있었던 거 아니야?,그렇게 기다리고 있었던 거 아니야?,DJSX20004853.json


성공적으로 말 더듬은 부분이 삭제된 것을 확인할 수 있다.

In [6]:
df.to_csv("datas/output/jeonla_dialect_data_processed_1.csv")

## 포스 태깅 분석

포스 태깅은 mecab 클래스로 진행한다. 실행 시간이 타 태깅 라이브러리에 비해 압도적으로 빠르기 때문이다.

In [3]:
import MeCab

def pos(sentence: str):
    """한국어 토큰을 분리합니다. 토큰과 품사를 튜플 리스트로 반환합니다.
        
        매개변수:
            sentence (str): 토큰화할 문장 
        반환값:
            token list (list[tuple]): 토큰과 품사 리스트
    """
    t = MeCab.Tagger()
    tag_result = t.parse(sentence)
    tag_result = tag_result.replace("\t", ".@!").replace("\n", ".@!").split(".@!")
    tag_word = tag_result[::2][:-1] # 마지막 EOS는 자른다
    tag_info = tag_result[1::2][:-1] # 마지막 EOS는 자른다
    return [(word, info.split(',')[0]) for word, info in zip(tag_word, tag_info)]

def morphs(sentence: str):
    """한국어 토큰을 분리합니다. 토큰의 리스트를 반환합니다.
        
        매개변수:
            sentence (str): 토큰화할 문장 
        반환값:
            token list (list): 토큰 리스트
    """
    t = MeCab.Tagger()
    tag_result = t.parse(sentence)
    tag_result = tag_result.replace("\t", ".@!").replace("\n", ".@!").split(".@!")
    return tag_result[::2][:-1]

def morph_and_preprocess(sentence: str):
    """한국어 토큰을 분리하고 전처리합니다. 토큰의 리스트를 반환합니다.
        
        매개변수:
            sentence (str): 토큰화할 문장 
        반환값:
            token list (list): 토큰 리스트
    """
    
    pos_result = pos(sentence)
    word_list = []
    for word, info in pos_result:
        if info == "NNP":
            # 고유명사일 경우 "고유"라는 단일 명사로 표현하도록 대체
            word_list.append("고유")
        else:
            word_list.append(word)
    
    return word_list

In [8]:
dialect_list = df["방언"].to_list()
dialect_list = list(map(morph_and_preprocess, dialect_list))
dialect_list

[['나', '는', '다이어트', '에', '대해서', '말', '하', '고', '싶', '어', '.'],
 ['하'],
 ['우리',
  '집',
  '은',
  '내',
  '가',
  '봤',
  '을',
  '때',
  '살찔',
  '수',
  '밖에',
  '없',
  '는',
  '구조',
  '를',
  '가지',
  '고',
  '있',
  '어',
  '.'],
 ['다이어트',
  '를',
  '해야',
  '되',
  '는',
  '사람',
  '은',
  '애초',
  '에',
  '야식',
  '을',
  '많이',
  '먹',
  '거나',
  '아',
  '어',
  '제도',
  '야식',
  '을',
  '먹',
  '었',
  '거든',
  '?'],
 ['그러니까',
  '다이어트',
  '는',
  '이게',
  '애초',
  '에',
  '조금',
  '만',
  '먹',
  '으면',
  '되',
  '는데',
  '많이',
  '먹',
  '으니까',
  '어쩔',
  '수',
  '없이',
  '하',
  '게',
  '될',
  '수',
  '밖',
  '에',
  '없',
  '어'],
 ['아', '나', '간헐', '적', '단식', '도', '해', '봤', '거든', '?'],
 ['하루', '한', '끼', '도', '해', '봤', '단', '말', '이', '야', '.'],
 ['근데',
  '하루',
  '한',
  '끼',
  '는',
  '우선',
  '옆',
  '에',
  '있',
  '는',
  '가족',
  '들',
  '이',
  '하루',
  '세',
  '끼',
  '를',
  '먹',
  '기',
  '때문',
  '에'],
 ['도저히', '할', '수', '가', '없', '어', '.'],
 ['밥', '을', '너무', '먹', '고', '싶', '어', '.'],
 ['그래서', '못', '해'],
 ['다이어트', '너무', '어려워', '그래서'],
 ['아',

In [ ]:
standard_list = df["표준어"].to_list()
standard_list = list(map(morph_and_preprocess, standard_list))
standard_list

### 형태소로 분리한 문장을 띄어쓰기로 구분하여 저장

In [ ]:
list_to_spaced_sentence = lambda li : " ".join(li)
df["방언"] = list(map(list_to_spaced_sentence, dialect_list))
df["표준어"] = list(map(list_to_spaced_sentence, standard_list))

df.head()

In [ ]:
df.to_csv("datas/output/jeonla_dialect_data_processed_2.csv")

### 단어 수 분석

### 표준어-방언 등장 단어 비교 분석